# Slice Sampling

Slice sampling {cite}`neal2003slice` is a Markov Chain Monte Carlo (MCMC) method for sampling from unnormalised probability distributions, developed by Radford Neal. Similarly to all other MCMC methods, Slice Sampling starts out from an initial point $x_0$ and evolves this according to a rule $p(x_{n + 1} | x_n)$, such that the distribution of $x_n$ approaches $\pi$ in the limit $n \to \infty$. The feature that makes Slice Sampling involves very few tunable parameters and virtually no critical design choices - unlike other sampling algorithms such as Metropolis Hastings, where the choice of proposal distribution can greatly affect the performance of the algorithm.

## Interval-expansion methods

### Step-out method

In [10]:
def step_out_expansion(x0, y, log_prob, w, m):
    
    U = np.random.rand()
    L = x0 - w * U
    R = L + w
    
    Lhist = [L]
    Rhist = [R]
    
    J = np.floor(m * np.random.rand())
    K = m - 1 - J
    
    fL = log_prob(L)
    fR = log_prob(R)
    
    while J > 0 and y <= fL:
        
        L = L - w
        J = J - 1
        fL = log_prob(L)
        
        Lhist.append(L)
        
    while K > 0 and y <= fR:
        
        R = R + w
        K = K - 1
        fR = log_prob(R)
        
        Rhist.append(R)
        
    return L, R, (Lhist, Rhist)

### Doubling method

In [11]:
def double_expansion(x0, y, log_prob, w, p):
    
    U = np.random.unform()
    L = x0 - w * U
    R = L + w
    K = p
    
    Lhist = [L]
    Rhist = [R]
    
    fL = log_prob(L)
    fR = log_prob(R)
    
    while K > 0 and (fL > y or fR > y):
        
        V = np.random.uniform()
        
        if V > 0.5:
            L = L - (R - L)
            fL = log_prob(L)
            
        else:
            R = R + (R - L)
            fR = log_prob(R)
        
        Lhist.append(L)
        Rhist.append(R)
        
        K = K - 1
            
    return L, R, (Lhist, Rhist)

## Interval sampling

### Sampling method

In [12]:
def sample_acceptable_set(x0, y, L, R, log_prob, method, params, shrinkage):
    
    L0, R0 = L, R
    
    while True:
        
        x = L + (R - L) * np.random.rand()
        
        accept = is_acceptable(x, y, L0, R0, log_prob, method, params)
        
        if accept:
            return x
        
        elif shrinkage and x <= x0:
            L = x
            
        elif shrinkage and x > x0:
            R = x

### Checking for acceptibility

In [13]:
def is_acceptable(x, y, L, R, log_prob, method, params):
    
    # Function value at candidate point
    f = log_prob(x)
    
    # For stepping out, check x is in the slice (y < f)
    if method == 'step_out':
        
        if y <= f:
            return True
        
        else:
            return False
    
    # For doubling, check the doubling sequence can be recreated the other way
    elif method == 'double':
        
        w, p = params
        
        fx = log_prob(x)
        
        while R - L > 1.5 * w:
            
            M = (R + L) / 2
            
            fL = log_prob(L)
            fM = log_prob(M)
            fR = log_prob(R)
            
            if x >= M and (y < fM or y < fR): L = M
                
            elif x < M and (y < fL or y < fM): R = M
            
            else: return False
            
        # If the point is not rejcected in the loop, it is acceptable
        return True

## The Slice Sampling algorithm

In [14]:
def sample(x0, log_prob, method, params, shrinkage):
    
    f0 = log_prob(x0)
    y = np.log(np.exp(f0) * np.random.rand())
    
    L = None
    R = None
    
    if method == 'step_out':
        w, m = params
        L, R, _ = step_out_expansion(x0, y, log_prob, w, m)
        
    elif method == 'double':
        w, p = params
        L, R, _ = double_expansion(x0, y, log_prob, w, p)
        
    x = sample_acceptable_set(x0=x0,
                              y=y,
                              L=L,
                              R=R,
                              log_prob=log_prob,
                              method=method,
                              params=params,
                              shrinkage=shrinkage)
    
    return x, y

## References

```{bibliography} ./references.bib
```